# Feature Store: Streaming ingestion SDK

**NOTE**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This notebook demonstrates how to use Vertex AI Feature Store's streaming ingestion at the SDK layer.

Learn more about [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore).

### Objective

Ingest features from a `Pandas DataFrame` into your Vertex AI Feature Store using `write_feature_values` method from the Vertex AI SDK.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Feature Store


The steps performed include:

- Create `Feature Store`
- Create new `Entity Type` for your `Feature Store`
- Ingest feature values from `Pandas DataFrame` into `Feature Store`'s `Entity Types`.

### Dataset

The dataset used for this notebook is the penguins dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). This dataset has the following features: `culmen_length_mm`, `culmen_depth_mm`, `flipper_length_mm`, `body_mass_g`, `species`, and `sex`.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


## Installation

Install the following packages required to execute this notebook.

In [1]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform\
                         google-cloud-bigquery\
                         numpy\
                         pandas\
                         db-dtypes\
                         pyarrow -q\
                         --user

**Note:** You can ignore the dependency and incompatibility errors.

### Restart the kernel

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [1]:
PROJECT_ID = "qwiklabs-gcp-01-9ce2f058dd08"  # Replace with your project-id

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [2]:
REGION = "us-central1"  # Replace with your region

### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [3]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Import libraries

In [4]:
import numpy as np
import pandas as pd
from google.cloud import aiplatform, bigquery

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [5]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## Download and prepare the data

In [6]:
def download_bq_table(bq_table_uri: str) -> pd.DataFrame:
    # Remove bq:// prefix if present
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix) :]

    table = bigquery.TableReference.from_string(bq_table_uri)

    # Create a BigQuery client
    bqclient = bigquery.Client(project=PROJECT_ID)

    # Download the table rows
    rows = bqclient.list_rows(
        table,
    )
    return rows.to_dataframe()

In [7]:
BQ_SOURCE = "bq://bigquery-public-data.ml_datasets.penguins"

# Download penguins BigQuery table
penguins_df = download_bq_table(BQ_SOURCE)

### Prepare the data

Feature values to be written to the Feature Store can take the form of a list of `WriteFeatureValuesPayload` objects, a Python `dict` of the form

`{entity_id : {feature_id : feature_value}, ...},`

or a pandas `Dataframe`, where the `index` column holds the unique entity ID strings and each remaining column represents a feature.  In this notebook, since you use a pandas `DataFrame` for ingesting features we convert the index column data type to `string` to be used as `Entity ID`.

In [8]:
# Prepare the data
penguins_df.index = penguins_df.index.map(str)

In [9]:
# Remove null values
NA_VALUES = ["NA", "."]
penguins_df = penguins_df.replace(to_replace=NA_VALUES, value=np.NaN).dropna()

## Create Feature Store and define schemas

Vertex AI Feature Store organizes resources hierarchically in the following order:

`Featurestore -> EntityType -> Feature`

You must create these resources before you can ingest data into Vertex AI Feature Store.

Learn more about [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore)

### Create a Feature Store

You create a Feature Store using `aiplatform.Featurestore.create` with the following parameters:

* `featurestore_id (str)`: The ID to use for this Featurestore, which will become the final component of the Featurestore's resource name. The value must be unique within the project and location.
* `online_store_fixed_node_count`: Configuration for online serving resources.
* `project`: Project to create EntityType in. If not set, project set in `aiplatform.init` is used.
* `location`: Location to create EntityType in. If not set, location set in `aiplatform.init` is used.
* `sync`:  Whether to execute this creation synchronously.

In [10]:
FEATURESTORE_ID = f"penguins_{UUID}"

penguins_feature_store = aiplatform.Featurestore.create(
    featurestore_id=FEATURESTORE_ID,
    online_store_fixed_node_count=1,
    project=PROJECT_ID,
    location=REGION,
    sync=True,
)

Creating Featurestore
Create Featurestore backing LRO: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/operations/2810940715230560256
Featurestore created. Resource name: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q
To use this Featurestore in another session:
featurestore = aiplatform.Featurestore('projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q')


##### Verify that the Feature Store is created
Check if the Feature Store was successfully created by running the following code block.

In [11]:
fs = aiplatform.Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=PROJECT_ID,
    location=REGION,
)
print(fs.gca_resource)

name: "projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q"
create_time {
  seconds: 1696760263
  nanos: 662073000
}
update_time {
  seconds: 1696760263
  nanos: 767578000
}
etag: "AMEw9yObiEB_LQqwBMC9Q2mI9Z-Oez_9KpDj-q-3pSCDBAUVBYDgVKK-NM_8-kNEHwqV"
online_serving_config {
  fixed_node_count: 1
}
state: STABLE
online_storage_ttl_days: 4000



### Create an EntityType

An entity type is a collection of semantically related features. You define your own entity types, based on the concepts that are relevant to your use case. For example, a movie service might have the entity types `movie` and `user`, which group related features that correspond to movies or users.

Here, you create an entity type entity type named `penguin_entity_type` using `create_entity_type` with the following parameters:
* `entity_type_id (str)`: The ID to use for the EntityType, which will become the final component of the EntityType's resource name. The value must be unique within a Feature Store.
* `description`: Description of the EntityType.

In [12]:
ENTITY_TYPE_ID = f"penguin_entity_type_{UUID}"

# Create penguin entity type
penguins_entity_type = penguins_feature_store.create_entity_type(
    entity_type_id=ENTITY_TYPE_ID,
    description="Penguins entity type",
)

Creating EntityType
Create EntityType backing LRO: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q/operations/3710534740797816832
EntityType created. Resource name: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q
To use this EntityType in another session:
entity_type = aiplatform.EntityType('projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q')


##### Verify that the EntityType is created
Check if the Entity Type was successfully created by running the following code block.

In [13]:
entity_type = penguins_feature_store.get_entity_type(entity_type_id=ENTITY_TYPE_ID)

print(entity_type.gca_resource)

name: "projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q"
description: "Penguins entity type"
create_time {
  seconds: 1696760276
  nanos: 154838000
}
update_time {
  seconds: 1696760276
  nanos: 154840000
}
etag: "AMEw9yMJTP0fZTCd6GONZvaxDxGeuoZRe9GLon91tV0d1vEwTB1zihskOd4YVkLUc-A="
monitoring_config {
}



### Create Features
A feature is a measurable property or attribute of an entity type. For example, `penguin` entity type has features such as `flipper_length_mm`, and `body_mass_g`. Features can be created within each entity type.

When you create a feature, you specify its value type such as `DOUBLE`, and `STRING`. This value determines what value types you can ingest for a particular feature.

Learn more about [Feature Value Types](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.featurestores.entityTypes.features)

In [14]:
penguins_feature_configs = {
    "species": {
        "value_type": "STRING",
    },
    "island": {
        "value_type": "STRING",
    },
    "culmen_length_mm": {
        "value_type": "DOUBLE",
    },
    "culmen_depth_mm": {
        "value_type": "DOUBLE",
    },
    "flipper_length_mm": {
        "value_type": "DOUBLE",
    },
    "body_mass_g": {"value_type": "DOUBLE"},
    "sex": {"value_type": "STRING"},
}

You can create features either using `create_feature` or `batch_create_features`. Here, for convinience, you have added all feature configs in one variabel, so we use `batch_create_features`.

In [15]:
penguin_features = penguins_entity_type.batch_create_features(
    feature_configs=penguins_feature_configs,
)

Batch creating features EntityType entityType: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q
Batch create Features EntityType entityType backing LRO: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q/operations/940820969964961792
EntityType entityType Batch created features. Resource name: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q


### Write features to the Feature Store
Use the `write_feature_values` API to write a feature to the Feature Store with the following parameter:

* `instances`: Feature values to be written to the Feature Store that can take the form of a list of WriteFeatureValuesPayload objects, a Python dict, or a pandas Dataframe.

This streaming ingestion feature has been introduced to the Vertex AI SDK under the **preview** namespace. Here, you pass the pandas `Dataframe` you created from penguins dataset as `instances` parameter.

Learn more about [Streaming ingestion API](https://github.com/googleapis/python-aiplatform/blob/e6933503d2d3a0f8a8f7ef8c178ed50a69ac2268/google/cloud/aiplatform/preview/featurestore/entity_type.py#L36)

In [16]:
penguins_entity_type.preview.write_feature_values(instances=penguins_df)

Writing EntityType feature values: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q
EntityType feature values written. Resource name: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q


resource name: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q/entityTypes/penguin_entity_type_jlayop4q

## Read back written features

Wait a few seconds for the write to propagate, then do an online read to confirm the write was successful.

In [17]:
ENTITY_IDS = [str(x) for x in range(100)]
penguins_entity_type.read(entity_ids=ENTITY_IDS)

,entity_id,island,body_mass_g,sex,culmen_depth_mm,culmen_length_mm,species,flipper_length_mm
0,0,Dream,3475.0,FEMALE,18.4,36.6,Adelie Penguin (Pygoscelis adeliae),184.0
1,1,Dream,4650.0,MALE,19.1,39.8,Adelie Penguin (Pygoscelis adeliae),184.0
2,10,Dream,4300.0,MALE,18.9,40.8,Adelie Penguin (Pygoscelis adeliae),208.0
3,11,Dream,3650.0,MALE,18.7,39.0,Adelie Penguin (Pygoscelis adeliae),185.0
4,12,Dream,3000.0,FEMALE,16.9,37.0,Adelie Penguin (Pygoscelis adeliae),185.0
...,...,...,...,...,...,...,...,...
95,95,Dream,3150.0,FEMALE,18.0,36.5,Adelie Penguin (Pygoscelis adeliae),182.0
96,96,Dream,3425.0,MALE,19.0,41.1,Adelie Penguin (Pygoscelis adeliae),182.0
97,97,Dream,3450.0,FEMALE,17.9,36.0,Adelie Penguin (Pygoscelis adeliae),190.0
98,98,Dream,3900.0,MALE,17.5,41.1,Adelie Penguin (Pygoscelis adeliae),190.0


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [18]:
penguins_feature_store.delete(force=True)

Deleting Featurestore : projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q
Delete Featurestore  backing LRO: projects/67667283155/locations/us-central1/operations/491586907134754816
Featurestore deleted. . Resource name: projects/67667283155/locations/us-central1/featurestores/penguins_jlayop4q
